# Imports

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from skimage import exposure
import os
import random


# Image Processingm Pipeline Function

In [ ]:
def process_fundus_image(input_path, output_path, save_grid=False, grid_output_path=None):
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: Unable to read image at {input_path}")
        return None
        
    
    # Convert to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if save_grid:
        # Create a figure to display all processing steps
        plt.figure(figsize=(20, 12))
        plt.subplot(3, 3, 1)
        plt.imshow(img_rgb)
        plt.title('Original Image')
        plt.axis('off')
    
    # 1. Enhanced Color Enhancement
    pil_img = Image.fromarray(img_rgb)
    enhancer = ImageEnhance.Color(pil_img)
    enhanced_img = enhancer.enhance(2.0)
    contrast_enhancer = ImageEnhance.Contrast(enhanced_img)
    enhanced_img = contrast_enhancer.enhance(1.3)
    brightness_enhancer = ImageEnhance.Brightness(enhanced_img)
    enhanced_img = brightness_enhancer.enhance(1.1)
    enhanced_array = np.array(enhanced_img)
    
    if save_grid:
        plt.subplot(3, 3, 2)
        plt.imshow(enhanced_array)
        plt.title('Color Enhanced')
        plt.axis('off')
    
    # 2. Green Channel Extraction
    green_channel = enhanced_array[:,:,1]
    
    if save_grid:
        plt.subplot(3, 3, 3)
        plt.imshow(green_channel, cmap='gray')
        plt.title('Green Channel')
        plt.axis('off')
    
    # 3. Improved Illumination Correction (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eq_green = clahe.apply(cv2.convertScaleAbs(green_channel))
    eq_green = eq_green.astype('float') / 255.0
    
    if save_grid:
        plt.subplot(3, 3, 4)
        plt.imshow(eq_green, cmap='gray')
        plt.title('Illumination Corrected')
        plt.axis('off')
    
    # 4. Noise Reduction (Bilateral Filter)
    eq_green_8bit = (eq_green * 255).astype(np.uint8)
    blurred = cv2.bilateralFilter(eq_green_8bit, 9, 75, 75)
    blurred = blurred.astype('float') / 255.0
    
    if save_grid:
        plt.subplot(3, 3, 5)
        plt.imshow(blurred, cmap='gray')
        plt.title('Noise Reduced')
        plt.axis('off')
    
    # 5. Enhanced Edge Enhancement (Sobel + Laplacian)
    sobelx = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    edges = (0.7 * sobel + 0.3 * np.abs(laplacian))
    edges = exposure.rescale_intensity(edges, out_range=(0, 1))
    
    if save_grid:
        plt.subplot(3, 3, 6)
        plt.imshow(edges, cmap='gray')
        plt.title('Edge Enhanced')
        plt.axis('off')
    
    # 6. Field of View Standardization (Create circular mask)
    h, w = img_rgb.shape[:2]
    center = (w // 2, h // 2)
    radius = min(center[0], center[1])
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.circle(mask, center, radius, 255, -1)
    masked_img = enhanced_array.copy()
    for i in range(3):
        masked_img[:,:,i] = cv2.bitwise_and(masked_img[:,:,i], mask)
    
    if save_grid:
        plt.subplot(3, 3, 7)
        plt.imshow(masked_img)
        plt.title('Field of View Standardized')
        plt.axis('off')
    
    # 7. Resolution Standardization
    standard_size = (512, 512)
    resized_img = cv2.resize(masked_img, standard_size)
    
    if save_grid:
        plt.subplot(3, 3, 8)
        plt.imshow(resized_img)
        plt.title('Resolution Standardized')
        plt.axis('off')
    
    # 8. Final Enhanced Image with dramatic green-orange contrast
    green_std = resized_img[:,:,1]
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    green_enhanced = clahe.apply(cv2.convertScaleAbs(green_std))
    edges_final = cv2.Laplacian(green_enhanced, cv2.CV_64F)
    edges_final = np.abs(edges_final)
    edges_final = exposure.rescale_intensity(edges_final, out_range=(0, 1))
    enhanced_final = np.zeros_like(resized_img)
    enhanced_final[:,:,0] = resized_img[:,:,0]
    enhanced_final[:,:,1] = np.clip(resized_img[:,:,1] * (1 + edges_final * 3.0), 0, 255)
    enhanced_final[:,:,2] = resized_img[:,:,2] * 0.7
    kernel = np.array([[-1,-1,-1], 
                       [-1, 9,-1],
                       [-1,-1,-1]])
    enhanced_final = cv2.filter2D(enhanced_final, -1, kernel)
    hsv = cv2.cvtColor(enhanced_final, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    h = h.astype(np.float32)
    h = np.where(edges_final > 0.2, 60, h)
    h = np.clip(h, 0, 179).astype(np.uint8)
    s = np.clip(s.astype(np.float32) * 1.5, 0, 255).astype(np.uint8)
    hsv_enhanced = cv2.merge([h, s, v])
    enhanced_final = cv2.cvtColor(hsv_enhanced, cv2.COLOR_HSV2RGB)
    
    if save_grid:
        plt.subplot(3, 3, 9)
        plt.imshow(enhanced_final)
        plt.title('Final Image - Superimposed')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(grid_output_path, dpi=300)
        print(f"Processing visualization saved to {grid_output_path}")
        plt.close()
    
    # Save the final processed image
    cv2.imwrite(output_path, cv2.cvtColor(enhanced_final, cv2.COLOR_RGB2BGR))
    print(f"Final processed image saved to {output_path}")
    
    return enhanced_final


# Folder Processing Functions

In [ ]:
def process_folder(input_dir, output_dir, grid_output_dir):
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(grid_output_dir, exist_ok=True)
    
    valid_extensions = ('.jpg')
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(valid_extensions)]
    
    if not image_files:
        print(f"No valid images found in {input_dir}")
        return

    grid_image = random.choice(image_files)
    print(f"Selected {grid_image} for processing step visualization")

    for filename in image_files:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, f"processed_{filename}")
        
        if filename == grid_image:
            grid_output_path = os.path.join(grid_output_dir, 'fundus_processing_steps.png')
            process_fundus_image(input_path, output_path, save_grid=True, grid_output_path=grid_output_path)
        else:
            process_fundus_image(input_path, output_path, save_grid=False)


# Run the Processing

In [ ]:
input_dir = 'input_images'  
output_dir = 'processed_output'  
grid_output_dir = 'new_output_grid'  

try:
    process_folder(input_dir, output_dir, grid_output_dir)
    print("All images processed successfully!")
except Exception as e:
    print(f"An error occurred during processing: {e}")
